## Topic Analysis of Review Data

#### DESCRIPTION

Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

#### Problem Statement: 

A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

**Domain:** Amazon reviews for a leading phone brand

#### Analysis to be done: 
POS tagging, topic modeling using LDA, and topic interpretation

**Dataset:** ‘K8 Reviews v0.2.csv’

**Sentiment:** The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)

**Reviews:** The main text of the review

#### Steps to perform:

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

#### Tasks: 

1. Read the .csv file using Pandas. Take a look at the top few records.

2. Normalize casings for the review text and extract the text into a list for easier manipulation.

3. Tokenize the reviews using NLTKs word_tokenize function.

4. Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

5. For the topic model, we should  want to include only nouns.

  - Find out all the POS tags that correspond to nouns.

  - Limit the data to only terms with these tags.

6. Lemmatize. 

  - Different forms of the terms need to be treated as one.

  - No need to provide POS tag to lemmatizer for now.

7. Remove stopwords and punctuation (if there are any). 

8. Create a topic model using LDA on the cleaned up data with 12 topics.

  - Print out the top terms for each topic.

  - What is the coherence of the model with the c_v metric?

9. Analyze the topics through the business lens.

  - Determine which of the topics can be combined.

10. Create topic model using LDA with what you think is the optimal number of topics

  - What is the coherence of the model?

11. The business should  be able to interpret the topics.

  - Name each of the identified topics.

  - Create a table with the topic name and the top 10 terms in each to present to the  business.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

#### 1. Read the data

In [2]:
data = pd.read_csv('K8 Reviews v0.2.csv')

In [3]:
data.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14675 entries, 0 to 14674
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  14675 non-null  int64 
 1   review     14675 non-null  object
dtypes: int64(1), object(1)
memory usage: 229.4+ KB


#### 2. Normalizing the casing to lower

In [5]:
data['review_cleaned'] = data['review'].apply(lambda review: review.lower())

In [6]:
data.head()

,sentiment,review,review_cleaned
0,1,Good but need updates and improvements,good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr...","worst mobile i have bought ever, battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...,when i will get my 10% cash back.... its alrea...
3,1,Good,good
4,0,The worst phone everThey have changed the last...,the worst phone everthey have changed the last...


#### 3. Tokenize the reviews using NLTKs word_tokenize function.

In [7]:
reviews = data['review_cleaned']
reviews = [nltk.word_tokenize(review) for review in reviews]

In [8]:
len(reviews)

14675

In [9]:
pprint(reviews[:5], compact=True)

[['good', 'but', 'need', 'updates', 'and', 'improvements'],
 ['worst', 'mobile', 'i', 'have', 'bought', 'ever', ',', 'battery', 'is',
  'draining', 'like', 'hell', ',', 'backup', 'is', 'only', '6', 'to', '7',
  'hours', 'with', 'internet', 'uses', ',', 'even', 'if', 'i', 'put', 'mobile',
  'idle', 'its', 'getting', 'discharged.this', 'is', 'biggest', 'lie', 'from',
  'amazon', '&', 'lenove', 'which', 'is', 'not', 'at', 'all', 'expected', ',',
  'they', 'are', 'making', 'full', 'by', 'saying', 'that', 'battery', 'is',
  '4000mah', '&', 'booster', 'charger', 'is', 'fake', ',', 'it', 'takes', 'at',
  'least', '4', 'to', '5', 'hours', 'to', 'be', 'fully', 'charged.do', "n't",
  'know', 'how', 'lenovo', 'will', 'survive', 'by', 'making', 'full', 'of',
  'us.please', 'don', ';', 't', 'go', 'for', 'this', 'else', 'you', 'will',
  'regret', 'like', 'me', '.'],
 ['when', 'i', 'will', 'get', 'my', '10', '%', 'cash', 'back', '...', '.',
  'its', 'already', '15', 'january..'],
 ['good'],
 ['the', 

#### 4. Perform parts-of-speech tagging on each document using the NLTK POS tagger

In [10]:
%%time
review_pos_tags = [nltk.pos_tag(doc) for doc in reviews]
pprint(review_pos_tags[:5], compact=True)

[[('good', 'JJ'), ('but', 'CC'), ('need', 'VBP'), ('updates', 'NNS'),
  ('and', 'CC'), ('improvements', 'NNS')],
 [('worst', 'JJS'), ('mobile', 'NN'), ('i', 'NN'), ('have', 'VBP'),
  ('bought', 'VBN'), ('ever', 'RB'), (',', ','), ('battery', 'NN'),
  ('is', 'VBZ'), ('draining', 'VBG'), ('like', 'IN'), ('hell', 'NN'),
  (',', ','), ('backup', 'NN'), ('is', 'VBZ'), ('only', 'RB'), ('6', 'CD'),
  ('to', 'TO'), ('7', 'CD'), ('hours', 'NNS'), ('with', 'IN'),
  ('internet', 'JJ'), ('uses', 'NNS'), (',', ','), ('even', 'RB'), ('if', 'IN'),
  ('i', 'JJ'), ('put', 'VBP'), ('mobile', 'JJ'), ('idle', 'NN'),
  ('its', 'PRP$'), ('getting', 'VBG'), ('discharged.this', 'NN'), ('is', 'VBZ'),
  ('biggest', 'JJS'), ('lie', 'NN'), ('from', 'IN'), ('amazon', 'NN'),
  ('&', 'CC'), ('lenove', 'NN'), ('which', 'WDT'), ('is', 'VBZ'), ('not', 'RB'),
  ('at', 'IN'), ('all', 'DT'), ('expected', 'VBN'), (',', ','), ('they', 'PRP'),
  ('are', 'VBP'), ('making', 'VBG'), ('full', 'JJ'), ('by', 'IN'),
  ('saying', 'V

#### 5. Including only nouns for building the Topic Model

In [11]:
review_nouns = [[token for token, pos in doc if pos.startswith('NN')] for doc in review_pos_tags]
pprint(review_nouns[:5], compact=True)

[['updates', 'improvements'],
 ['mobile', 'i', 'battery', 'hell', 'backup', 'hours', 'uses', 'idle',
  'discharged.this', 'lie', 'amazon', 'lenove', 'battery', 'charger', 'hours',
  'don'],
 ['i', '%', 'cash', 'january..'], [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]


#### 6. Lemmatize

In [12]:
%%time
review_lemmatized = [[nltk.stem.WordNetLemmatizer().lemmatize(token) for token in doc] for doc in review_nouns]
pprint(review_lemmatized[:5])

[['update', 'improvement'],
 ['mobile',
  'i',
  'battery',
  'hell',
  'backup',
  'hour',
  'us',
  'idle',
  'discharged.this',
  'lie',
  'amazon',
  'lenove',
  'battery',
  'charger',
  'hour',
  'don'],
 ['i', '%', 'cash', 'january..'],
 [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]
Wall time: 3.13 s


#### 7. Remove stopwords and punctuation (if there are any).

In [13]:
%%time
review_no_sw = [[token for token in doc if token not in nltk.corpus.stopwords.words('english') 
                 and token.isalpha()] 
                for doc in review_lemmatized]
pprint(review_no_sw[:5])

[['update', 'improvement'],
 ['mobile',
  'battery',
  'hell',
  'backup',
  'hour',
  'us',
  'idle',
  'lie',
  'amazon',
  'lenove',
  'battery',
  'charger',
  'hour'],
 ['cash'],
 [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]
Wall time: 25.9 s


#### 8. Create a topic model using LDA on the cleaned up data with 12 topics.

In [14]:
# Build a Dictionary - association word to numeric id
dictionary = gensim.corpora.Dictionary(review_no_sw)
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in review_no_sw]

In [15]:
NUM_TOPICS = 12
lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

#### 8.1 Print out the top terms for each topic.

In [16]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    pprint("Topic #{}: {}".format(idx, lda_model.print_topic(idx, 10)), compact=True)

LDA Model:
('Topic #0: 0.147*"battery" + 0.078*"phone" + 0.050*"heat" + 0.037*"charger" + '
 '0.034*"day" + 0.030*"h" + 0.024*"life" + 0.024*"hour" + 0.022*"charge" + '
 '0.020*"month"')
('Topic #1: 0.138*"issue" + 0.091*"network" + 0.043*"problem" + 0.026*"video" '
 '+ 0.021*"sim" + 0.020*"phone" + 0.019*"bit" + 0.016*"quality" + '
 '0.015*"battery" + 0.015*"camera"')
('Topic #2: 0.095*"note" + 0.080*"lenovo" + 0.056*"hai" + 0.043*"smartphone" + '
 '0.034*"box" + 0.030*"ok" + 0.028*"phone" + 0.019*"killer" + 0.016*"image" + '
 '0.014*"today"')
('Topic #3: 0.075*"device" + 0.065*"phone" + 0.038*"return" + 0.033*"amazon" + '
 '0.023*"headphone" + 0.023*"service" + 0.022*"problem" + 0.021*"budget" + '
 '0.020*"volta" + 0.018*"policy"')
('Topic #4: 0.154*"price" + 0.110*"phone" + 0.082*"feature" + 0.061*"range" + '
 '0.032*"handset" + 0.027*"speaker" + 0.025*"experience" + 0.023*"everything" '
 '+ 0.019*"product" + 0.019*"note"')
('Topic #5: 0.230*"phone" + 0.077*"camera" + 0.044*"battery

In [17]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.129368 -0.020027       1        1  18.568121
10    -0.090972  0.065111       2        1  14.599727
8     -0.042605 -0.204411       3        1  10.364258
11    -0.112832  0.015935       4        1   9.600409
0     -0.136909 -0.054433       5        1   9.004350
1     -0.092236 -0.058937       6        1   6.716942
3      0.040885  0.150055       7        1   6.025668
4      0.054668  0.135012       8        1   5.897274
9      0.081919 -0.052715       9        1   5.475067
6      0.312058 -0.167219      10        1   5.285929
7     -0.013002 -0.013532      11        1   4.437072
2      0.128394  0.205161      12        1   4.025183, topic_info=        Term         Freq        Total Category  logprob  loglift
47   product  2259.000000  2259.000000  Default  30.0000  30.0000
11    mobile  1502.000000  1502.000000  Default  29.0000  29.0000
55    camera  3181.000000  3181.000000  Default  28.0000  28.0000
63     price   914.000000   914.000000  Default  27.0000  27.0000
4    battery  3123.000000  3123.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
15     phone    90.707161  6954.558594  Topic12  -3.5733  -1.1269
304       se    22.123783    38.512135  Topic12  -4.9843   2.6583
117       tv    22.094582    60.705753  Topic12  -4.9856   2.2019
169     look    23.052591   148.635773  Topic12  -4.9432   1.3489
90   quality    25.165266  1392.118408  Topic12  -4.8555  -0.8005

[725 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1367      1  0.051941        aap
1367     12  0.934937        aap
1328      1  0.972639     access
332       3  0.051549  accessory
332       7  0.902106  accessory
...     ...       ...        ...
1448      8  0.857278     xiaomi
305       7  0.692201         ye
305      12  0.276881         ye
1024      2  0.972823  yesterday
1770      3  0.967436       zoom

[1874 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 11, 9, 12, 1, 2, 4, 5, 10, 7, 8, 3])

#### 8.2 coherence of the model with the c_v metric

In [18]:
# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=review_no_sw, dictionary=dictionary, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5134232511606582


#### 9. Analyze the topics through the business lens - Determine which of the topics can be combined

Now since the problem at hand demands identifying the diffrent aspects of the budget smartphone launched in the market, it is imperative that the tokens (words in the customer reviews) like 'phone', 'mobile' etc. or sentimental keys like 'problem', 'issue' may be removed from the topic model. That way topic model focuses on the variety of aspects like 'camera', 'battery'.

In [19]:
custom_stop_words = ['phone', 'problem', 'issue', 'product', 'mobile']

review_no_custom_sw = [[token for token in doc if token not in custom_stop_words] for doc in review_no_sw]

pprint(review_no_custom_sw[:5])

[['update', 'improvement'],
 ['battery',
  'hell',
  'backup',
  'hour',
  'us',
  'idle',
  'lie',
  'amazon',
  'lenove',
  'battery',
  'charger',
  'hour'],
 ['cash'],
 [],
 ['everthey', 'amazon', 'amazon']]


#### 10. Create topic model using LDA with what you think is the optimal number of topics

We perform a GridSearch over the parameter space of NUM_TOPICS and get the best cv score

In [42]:
review_optimised = []
for doc in review_no_custom_sw:
    tokens = ""
    for token in doc:
        tokens = tokens + " " + token
    review_optimised.append(tokens[1:])

review_optimised[:5]

['update improvement',
 'battery hell backup hour us idle lie amazon lenove battery charger hour',
 'cash',
 '',
 'everthey amazon amazon']

In [43]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=False,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(review_optimised)

In [44]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.5335390057096686 %


In [47]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=12,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=12, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [48]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -302316.9190399189
Perplexity:  302.07201970206717
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 12,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [51]:
# Define Search Param
search_params = {'n_components': [10, 11, 12]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

#### 10.1 Coherence of the optimal model

In [52]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'n_components': 10}
Best Log Likelihood Score:  -64122.96238764863
Model Perplexity:  254.16612932482374


#### 11. Business interpretation of the identified topics

#### 11.1. Table with the topic names and the top 10 terms in each to present to the business